In [19]:
import jax
from jax import numpy as jnp
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
from pathlib import Path
import lovely_jax as lj
lj.monkey_patch()

In [2]:
import pynvml
def get_memory_free_MiB(gpu_index):
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(int(gpu_index))
    mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    return mem_info.free // 1024 ** 2

get_memory_free_MiB(0)

11176

In [73]:
from jax import grad, jit, vmap
from jax import random

In [125]:
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(jnp.array, [x_train, y_train, x_valid, y_valid])

In [10]:
x_train

Array[50000, 784] n=39200000 x∈[0., 0.996] μ=0.130 σ=0.307 gpu:0

## The forward and backward passes

## Foundations version

### Basic architecture

In [94]:
random.normal(random.PRNGKey(42), (50,10))*1e-2

Array[50, 10] n=500 x∈[-0.028, 0.027] μ=-4.746e-06 σ=0.010 gpu:0

In [96]:
# A helper function to randomly initialize weights and biases
# for a dense neural network layer
def random_layer_params(m, n, key, scale=1):
  w_key, b_key = random.split(key)
  return scale * random.normal(w_key, (m, n)), scale * random.normal(b_key, (n,))

random_layer_params(784,50, random.PRNGKey(42))

(Array[784, 50] n=39200 x∈[-3.969, 4.653] μ=0.001 σ=1.008 gpu:0,
 Array[50] x∈[-2.799, 1.671] μ=-0.163 σ=1.007 gpu:0)

In [98]:
  sizes = [784, 512, 512, 10]
  keys = random.split(random.PRNGKey(42), len(sizes))
  keys

Array[4, 2] u32 n=8 x∈[13772463, 3276725750] μ=1.549e+09 σ=1.162e+09 gpu:0 [[2954079971, 2575172198], [1139032965, 1035738655], [3276725750, 1223413151], [173471767, 13772463]]

In [99]:
[(m, n) for m, n in zip(sizes[:-1], sizes[1:])]

[(784, 512), (512, 512), (512, 10)]

In [100]:
# Initialize all layers for a fully-connected neural network with sizes "sizes"
def init_network_params(sizes, key):
  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

In [102]:
params = init_network_params(sizes, random.PRNGKey(42))
params

[(Array[784, 512] n=401408 x∈[-4.272, 5.061] μ=-0.000 σ=0.999 gpu:0,
  Array[512] x∈[-2.752, 2.637] μ=0.017 σ=1.006 gpu:0),
 (Array[512, 512] n=262144 x∈[-4.694, 4.418] μ=0.003 σ=0.999 gpu:0,
  Array[512] x∈[-3.008, 2.639] μ=0.027 σ=0.996 gpu:0),
 (Array[512, 10] n=5120 x∈[-3.469, 4.165] μ=0.017 σ=1.001 gpu:0,
  Array[10] x∈[-1.779, 1.051] μ=-0.887 σ=0.832 gpu:0 [-0.804, -1.676, -1.779, -1.359, 1.051, -0.639, -0.035, -1.232, -0.742, -1.655])]

In [103]:
from jax.scipy.special import logsumexp

def relu(x): return jnp.maximum(0, x)

def predict(params, image):
  # per-example predictions
  activations = image
  for w, b in params[:-1]:
    outputs = activations@w + b
    activations = relu(outputs)
  
  final_w, final_b = params[-1]
  logits = activations@final_w + final_b
  return logits - logsumexp(logits)

In [104]:
# This works on single examples
random_flattened_image = random.normal(random.PRNGKey(1), (28 * 28,))
preds = predict(params, random_flattened_image)
preds

Array[10] x∈[-2.107e+04, 0.] μ=-1.238e+04 σ=6.829e+03 gpu:0 [-1.162e+04, -1.724e+04, -2.047e+04, 0., -1.182e+04, -1.465e+04, -1.250e+04, -2.107e+04, -535.592, -1.385e+04]

In [105]:
batched_predict = vmap(predict, in_axes=(None, 0))

In [106]:
random_flattened_images = random.normal(random.PRNGKey(1), (10, 28 * 28))
batched_preds = batched_predict(params, random_flattened_images)
batched_preds

Array[10, 10] n=100 x∈[-2.639e+04, 0.] μ=-1.145e+04 σ=7.787e+03 gpu:0

In [107]:
a = random.normal(random.PRNGKey(42), (50,10))
b = random.normal(random.PRNGKey(43), (10,10))
a,b

(Array[50, 10] n=500 x∈[-2.774, 2.730] μ=-0.000 σ=1.033 gpu:0,
 Array[10, 10] n=100 x∈[-2.508, 2.865] μ=0.049 σ=0.985 gpu:0)

In [113]:
a@b, jnp.dot(a,b)

(Array[50, 10] n=500 x∈[-11.963, 9.819] μ=0.010 σ=3.019 gpu:0,
 Array[50, 10] n=500 x∈[-11.963, 9.819] μ=0.010 σ=3.019 gpu:0)

In [114]:
jnp.arange(10)

Array[10] i32 x∈[0, 9] μ=4.500 σ=2.872 gpu:0 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [117]:
a = jnp.array([10,20,30])
a[:,None]

Array[3, 1] i32 x∈[10, 30] μ=20.000 σ=8.165 gpu:0 [[10], [20], [30]]

In [118]:
def one_hot(y, k, dtype=jnp.float32):
  """Create a one-hot encoding of y of size k klasses"""
  return jnp.array(y[:, None] == jnp.arange(k), dtype)


In [142]:
def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == target_class)

In [128]:
from functools import partial

In [130]:
train_labels, test_labels = map(partial(one_hot, k=10), (y_train, y_valid))
train_images, test_images = x_train, x_valid
train_images, train_labels, test_images, test_labels

(Array[50000, 784] n=39200000 x∈[0., 0.996] μ=0.130 σ=0.307 gpu:0,
 Array[50000, 10] n=500000 x∈[0., 1.000] μ=0.100 σ=0.300 gpu:0,
 Array[10000, 784] n=7840000 x∈[0., 0.996] μ=0.129 σ=0.305 gpu:0,
 Array[10000, 10] n=100000 x∈[0., 1.000] μ=0.100 σ=0.300 gpu:0)

In [135]:
train_labels-train_labels

Array[50000, 10] all_zeros gpu:0

In [131]:
jnp.mean(train_labels -train_labels)

Array gpu:0 0.100

In [136]:
def mse(o, t): return jnp.mean((o-t)**2)

In [137]:
def loss(params, images, targets): return mse(batched_predict(params, images), targets)

In [140]:
loss(params, train_images, train_labels), grad(loss)(params, train_images, train_labels)

(Array gpu:0 3.334e+07,
 [(Array[784, 512] n=401408 x∈[-1.781e+05, 2.268e+05] μ=1.916e+03 σ=1.738e+04 gpu:0,
   Array[512] x∈[-2.839e+05, 3.672e+05] μ=1.335e+04 σ=7.398e+04 gpu:0),
  (Array[512, 512] n=262144 x∈[-5.188e+05, 5.030e+05] μ=2.646e+03 σ=2.896e+04 gpu:0,
   Array[512] x∈[-2.165e+04, 2.077e+04] μ=675.916 σ=5.403e+03 gpu:0),
  (Array[512, 10] n=5120 x∈[-5.433e+05, 2.726e+06] μ=0.000 σ=2.679e+05 gpu:0,
   Array[10] x∈[-1.723e+03, 8.635e+03] μ=7.324e-05 σ=2.927e+03 gpu:0 [-967.755, -919.111, -1.242e+03, 412.967, -743.843, -922.010, -1.197e+03, -1.334e+03, 8.635e+03, -1.723e+03])])

In [141]:
step_size = 0.01
@jit
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]

In [143]:
num_epochs = 8
n_targets = 10
# batch_size = 128
import time

for epoch in range(num_epochs):
  start_time = time.time()
  x, y = train_images, train_labels
  params = update(params, x, y)
  epoch_time = time.time() - start_time

  train_acc = accuracy(params, train_images, train_labels)
  test_acc = accuracy(params, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))

Epoch 0 in 2.34 sec
Training set accuracy 0.09975999593734741
Test set accuracy 0.09609999507665634
Epoch 1 in 0.00 sec
Training set accuracy 0.09683999419212341
Test set accuracy 0.10089999437332153
Epoch 2 in 0.00 sec
Training set accuracy 0.09975999593734741
Test set accuracy 0.09609999507665634
Epoch 3 in 0.00 sec
Training set accuracy 0.09863999485969543
Test set accuracy 0.09910000115633011
Epoch 4 in 0.00 sec
Training set accuracy 0.09863999485969543
Test set accuracy 0.09910000115633011
Epoch 5 in 0.00 sec
Training set accuracy 0.09863999485969543
Test set accuracy 0.09910000115633011
Epoch 6 in 0.00 sec
Training set accuracy 0.09863999485969543
Test set accuracy 0.09910000115633011
Epoch 7 in 0.00 sec
Training set accuracy 0.09863999485969543
Test set accuracy 0.09910000115633011


In [21]:

n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, Array i32 gpu:0 10)

In [17]:
# num hidden
nh = 50

In [23]:
jax.random.normal(key, (m,nh), dtype=jax.numpy.float32)

Array[784, 50] n=39200 x∈[-4.057, 4.385] μ=0.004 σ=0.999 gpu:0

In [24]:
w1 = jax.random.normal(key, (m,nh), dtype=jax.numpy.float32)
b1 = jnp.zeros(nh)
w2 = jax.random.normal(key, (nh,1), dtype=jax.numpy.float32)
b2 = jnp.zeros(1)

In [25]:
def lin(x, w, b): return x@w + b

In [28]:
x_valid, w1, b1

(Array[10000, 784] n=7840000 x∈[0., 0.996] μ=0.129 σ=0.305 gpu:0,
 Array[784, 50] n=39200 x∈[-4.057, 4.385] μ=0.004 σ=0.999 gpu:0,
 Array[50] all_zeros gpu:0)

In [27]:
t = lin(x_valid, w1, b1)
t

Array[10000, 50] n=500000 x∈[-39.335, 46.460] μ=1.220 σ=9.081 gpu:0

In [72]:
def relu(x): return jnp.maximum(0, x)

In [33]:
t = relu(t)
t

Array[10000, 50] n=500000 x∈[0., 46.460] μ=4.220 σ=5.810 gpu:0

In [34]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [35]:
res = model(x_valid)
res

Array[10000, 1] x∈[-154.850, 80.024] μ=-31.277 σ=34.980 gpu:0

### Loss function: MSE

(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [37]:
res.shape,y_valid.shape

((10000, 1), (10000,))

In [38]:
(res-y_valid).shape

(10000, 10000)

We need to get rid of that trailing (,1), in order to use `mse`.

In [39]:
res[:,0].shape

(10000,)

In [40]:
res.squeeze().shape

(10000,)

In [41]:
(res[:,0]-y_valid).shape

(10000,)

In [46]:
jnp.float32(y_train)

dtype('float32')

In [49]:
y_train,y_valid = map(jnp.float32, (y_train,y_valid))

preds = model(x_train)
preds

Array[50000, 1] x∈[-186.163, 77.793] μ=-31.938 σ=36.011 gpu:0

In [52]:
def mse(output, targ): return ((output[:,0]-targ)** 2).mean()

In [53]:
mse(preds, y_train)

Array gpu:0 2.633e+03

In [65]:
sizes = [784, 512, 512, 10]
sizes[:-1], sizes[1:]

([784, 512, 512], [512, 512, 10])

In [71]:
list(zip(sizes[:-1], sizes[1:]))

[(784, 512), (512, 512), (512, 10)]

In [ ]:
# Global state which holds the parameters for the transformed function.
# get_param uses this to know where to get params from.
current_params = []

def transform(f):

  def apply_f(params, *args, **kwargs):
    current_params.append(params)
    outs = f(*args, **kwargs)
    current_params.pop()
    return outs

  return apply_f


def get_param(identifier):
  return current_params[-1][identifier]

### Gradients and backward pass

In [54]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [89]:
diff(3*x**2+9, x)

6*x

In [ ]:
# def lin(x, w, b): return x@w + b

In [62]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [63]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:,0]-targ
    loss = (res ** 2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [64]:
forward_and_backward(x_train, y_train)

TypeError: dot_general requires contracting dimensions to have the same shape, got (1,) and (50,).

In [ ]:
# Save for testing against later
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

We cheat a little bit and use PyTorch autograd to check our results.

In [ ]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

In [ ]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [ ]:
loss = forward(xt2, y_train)
loss.backward()

In [ ]:
for a,b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

## Refactor model

### Layers as classes

In [ ]:
# Global state which holds the parameters for the transformed function.
# get_param uses this to know where to get params from.
current_params = []

def transform(f):

  def apply_f(params, *args, **kwargs):
    current_params.append(params)
    outs = f(*args, **kwargs)
    current_params.pop()
    return outs

  return apply_f


def get_param(identifier):
  return current_params[-1][identifier]

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [ ]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b

    def __call__(self, inp):
        self.inp = inp
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp,self.targ = inp,targ
        self.out = mse(inp, targ)
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
loss = model(x_train, y_train)

In [ ]:
model.backward()

In [ ]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Module.forward()

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
loss = model(x_train, y_train)

In [ ]:
model.backward()

In [ ]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [ ]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [ ]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])